In [22]:


import pyarrow as pa
import numpy as np

from arrow_ext import ext


a = pa.array(["1", "1", "3", None, None])
b = pa.array([0, 1, 2, 1, 2], type=pa.int32())

tb = pa.table([a,b],names=["a","b"])

tb.to_pandas()




,a,b
0,1,0
1,1,1
2,3,2
3,None,1
4,None,2


In [6]:
tb.take(ext.getUniqueRowIndex(a)).to_pandas()

,a,b
0,1,0
1,3,2
2,None,1


In [7]:
tb.take(ext.getUniqueRowIndex(tb['a'].combine_chunks())).to_pandas()

,a,b
0,1,0
1,3,2
2,None,1


In [7]:
tb.filter(ext.duplicatesFilter(a)).to_pandas()

,a,b
0,1,0
1,3,2


In [4]:

tb.filter(ext.duplicatesFilter(tb['a'].combine_chunks())).to_pandas()

,a,b
0,1,0
1,3,2


# speed test

In [5]:


import pyarrow as pa
import numpy as np

from arrow_ext import ext


import pyarrow.compute as pc
b = pa.array(np.random.randint(0,5000000,10000000), type=pa.int32())
c = pc.cast(b, pa.string())


/home/zhewu/.local/lib/python3.8/site-packages/pyarrow/libarrow.so /home/zhewu/.local/lib/python3.8/site-packages/pyarrow/libarrow_python.so
256


In [1]:

from arrow_ext import ext

/home/zhewu/.local/lib/python3.8/site-packages/pyarrow/libarrow.so /home/zhewu/.local/lib/python3.8/site-packages/pyarrow/libarrow_python.so
256


In [ ]:
ext.g

In [2]:


import pyarrow as pa
import numpy as np

from arrow_ext import ext


import pyarrow.compute as pc
b = pa.array(np.random.randint(0,50000,100000), type=pa.int32())
c = pc.cast(b, pa.string())

/home/zhewu/.local/lib/python3.8/site-packages/pyarrow/libarrow.so /home/zhewu/.local/lib/python3.8/site-packages/pyarrow/libarrow_python.so
256


In [2]:
%%time
f = ext.duplicatesFilter2(c)

CPU times: user 26.2 ms, sys: 7.65 ms, total: 33.8 ms
Wall time: 32.4 ms


In [3]:
%%time
f1 = ext.getUniqueRowIndex2(c,c)

In [1]:
f1

NameError: name 'f1' is not defined

In [7]:
f1 = ext.getUniqueRowIndex(c)

In [8]:
f1

[
  0,
  1,
  2,
  4,
  7
]

In [16]:
f1

[
  0,
  1,
  3,
  8
]

In [2]:
%%time
f2 = ext.getUniqueRowIndex(c)

CPU times: user 55.4 ms, sys: 3.97 ms, total: 59.4 ms
Wall time: 57.7 ms


In [1]:
f2

NameError: name 'f2' is not defined